In [94]:
import collections
import re
from matplotlib import pyplot as plt
from d2l.torch import d2l

In [95]:
def read_txt():
    with open('../../datasets/timemachine.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        return [re.sub(r'[^A-Za-z]+', ' ', line).strip().lower() for line in lines]


lines = read_txt()
print(lines)
print(len(lines))

print(lines[0])
print(lines[10])

['the project gutenberg ebook of the time machine by h g wells', '', 'this ebook is for the use of anyone anywhere in the united states and', 'most other parts of the world at no cost and with almost no restrictions', 'whatsoever you may copy it give it away or re use it under the terms', 'of the project gutenberg license included with this ebook or online at', 'www gutenberg org if you are not located in the united states you', 'will have to check the laws of the country where you are located before', 'using this ebook', '', 'title the time machine', '', 'author h g wells', '', 'release date july ebook', 'most recently updated october', '', 'language english', '', 'character set encoding utf', '', '', 'start of the project gutenberg ebook the time machine', '', '', '', '', 'the time machine', '', 'an invention', '', 'by h g wells', '', '', 'contents', '', 'i introduction', 'ii the machine', 'iii the time traveller returns', 'iv time travelling', 'v in the golden age', 'vi the sunset o

In [96]:
# tokenize

def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('wrong token：' + token)


tokens = tokenize(lines)
print(tokens[:10])
tokens_d2l = d2l.tokenize(lines)
# print(tokens_d2l[:10])
print(tokens == tokens_d2l)

[['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells'], [], ['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and'], ['most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions'], ['whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms'], ['of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at'], ['www', 'gutenberg', 'org', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you'], ['will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before'], ['using', 'this', 'ebook'], []]
True


In [106]:
# 计数器 Demo

a = collections.Counter(['a', 'b', 'c', 'c', 'c', 'b'])
a = sorted(a.items(), key=lambda x: x[1], reverse=True)
print('count:', a)

count: [('c', 3), ('b', 2), ('a', 1)]


In [98]:
# 统计词频
def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)


counter = count_corpus(tokens)
# counter

In [99]:
class Vocab:
    def __init__(self,
                 tokens=None,
                 min_freq=0,
                 reserved_token=None,
                 ):
        if tokens is None:
            tokens = []
        if reserved_token is None:
            reserved_token = []

        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # unk
        self.idx_to_token = ['<unk>'] + reserved_token
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [100]:
vocab = Vocab(tokens)
v = list(vocab.token_to_idx.items())[:10]
print(v)

vocab_d2l = d2l.Vocab(tokens)
v_d = list(vocab_d2l.token_to_idx.items())[:10]
# print(v_d)
print(v == v_d)

[('<unk>', 0), ('the', 1), ('and', 2), ('of', 3), ('i', 4), ('a', 5), ('to', 6), ('in', 7), ('was', 8), ('that', 9)]
True


In [101]:
for i in [0, 10]:
    print(tokens[i])
    print(vocab[tokens[i]])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[1, 53, 44, 314, 3, 1, 19, 46, 33, 1163, 1164, 360]
['title', 'the', 'time', 'machine']
[2445, 1, 19, 46]


In [102]:
def load_corpus_time_machine(max_tokens=-1, token='char'):
    lines = read_txt()
    tokens = tokenize(lines, token)
    vocab = Vocab(tokens)

    corpus = [vocab[token] for token in tokens]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab


corpus, vocab = load_corpus_time_machine(token='word')
len(corpus), len(vocab)

(3557, 4942)

In [103]:
freqs = [freq for token, freq in vocab.token_freqs]

print(freqs)

[2477, 1312, 1286, 1268, 877, 766, 606, 554, 458, 452, 441, 354, 281, 281, 264, 257, 247, 212, 211, 209, 199, 162, 158, 148, 142, 137, 136, 134, 129, 129, 129, 128, 126, 126, 124, 120, 115, 114, 114, 114, 112, 107, 102, 98, 95, 93, 93, 92, 92, 91, 91, 89, 88, 88, 87, 86, 86, 86, 79, 78, 77, 77, 75, 75, 75, 74, 74, 74, 72, 72, 72, 71, 71, 66, 65, 62, 61, 61, 60, 60, 59, 58, 57, 57, 57, 55, 54, 53, 53, 53, 52, 52, 51, 51, 49, 49, 49, 49, 48, 48, 48, 48, 47, 47, 47, 47, 46, 46, 45, 43, 43, 42, 41, 40, 40, 40, 40, 40, 39, 39, 39, 39, 38, 38, 38, 38, 37, 37, 37, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35, 35, 34, 34, 34, 33, 33, 33, 33, 32, 32, 32, 32, 32, 32, 31, 31, 31, 31, 31, 30, 30, 30, 29, 29, 28, 28, 28, 28, 28, 28, 28, 27, 27, 27, 27, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 

In [104]:
from plotly import graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(freqs))), y=freqs))
fig.layout = dict(
    xaxis=dict(type='log'),
    yaxis=dict(type='log')
)
fig.show()

In [137]:
corpus = [token for line in tokens for token in line]

# bigram
bigram_tokens = [pair for pair in zip(corpus[:-1], corpus[1:])]
bigram_vocab = Vocab(bigram_tokens)
bigram_freqs = [freq for token, freq in bigram_vocab.token_freqs]

# trigram
trigram_tokens = [pair for pair in zip(corpus[:-2], corpus[1:-1], corpus[2:])]
trigram_vocab = d2l.Vocab(trigram_tokens)
trigram_freqs = [freq for token, freq in trigram_vocab.token_freqs]

In [136]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(freqs))), y=freqs, name='unigram'))
fig.add_trace(go.Scatter(x=list(range(len(bigram_freqs))), y=bigram_freqs, name='bigram'))
fig.add_trace(go.Scatter(x=list(range(len(trigram_freqs))), y=trigram_freqs, name='trigram'))
fig.layout = dict(
    xaxis=dict(type='log'),
    yaxis=dict(type='log')
)
fig.show()